In [15]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pandas as pd
from IPython.display import Markdown, display,HTML,Image
from selenium import webdriver
import time
from pandas.plotting import table 
import matplotlib.pyplot as plt
import string

# Borrowed the offset idea from this (much more professionally done) tableau page
# https://public.tableau.com/app/profile/heidi.stockton/viz/PuzzlesofJaneStreet/JaneStreet
# No adjustments for similar names etc (apologies)

In [2]:
# get the solution links
pages = ['https://www.janestreet.com/puzzles/archive/index.html']+['https://www.janestreet.com/puzzles/archive/page'+str(i)+'/index.html' for i in range(2,13)]
links = [] # ['https://www.janestreet.com/puzzles/current-puzzle/']
for url in pages:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    links += ['https://www.janestreet.com/'+i['href'] for i in soup.find_all('a', {'class' :'solution-link'})]   

In [3]:
# scrape the names of solvers
participants = []
order = 1
driver = webdriver.Firefox()
for url in links:
    driver.get(url)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    participants += [[str(url).split("/")[-1],i.split(" (")[0],order] for i in soup.find_all('p', {'class' :'correct-submissions'})[0].stripped_strings]
    order +=1

In [17]:
outputs = []
for newcomer_offset in [5]:
    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df['Name']=df['Name'].str.lstrip("1234567890. ")
    df['Name'] = df['Name'].str.title()
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempts']
    df2['Score']=df2['Solved']/(df2['Attempts']+ newcomer_offset)
   
df2.sort_values(['Score','Solved'],ascending=False)[:15]

,Solved,Attempts,Score
Name,,,
Evan Semet,34,34,0.871795
Karl Mahlburg,51,58,0.809524
Senthil Rajasekaran,57,69,0.770270
Gareth Owen,40,49,0.740741
Aaditya Raghavan,42,52,0.736842
Blaine Hill,14,14,0.736842
Lazar Ilic,37,46,0.725490
Danica Xiong,15,16,0.714286
Calvin Pozderac,60,82,0.689655


In [19]:
df2.sort_values('Solved',ascending=False)[:15]

,Solved,Attempts,Score
Name,,,
Calvin Pozderac,60,82,0.689655
Senthil Rajasekaran,57,69,0.770270
Sean Egan,51,76,0.629630
Karl Mahlburg,51,58,0.809524
Keith Schneider,46,73,0.589744
Aaditya Raghavan,42,52,0.736842
Gareth Owen,40,49,0.740741
Heidi Stockton,38,95,0.380000
Sébastien Geeraert,37,67,0.513889


In [6]:
import xlwings as xw

sht = xw.Book().sheets[0]
sht.range('a1').options(index=False).value =  df